# Preprocessing training data

In [ ]:
# Import libararies
import os
import copy
import pandas as pd
import numpy as np
from collections import Counter
from datetime import date, timedelta
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Importing datasets
met_stations = pd.read_csv('/content/drive/MyDrive/FYP/met-stations.tsv',sep='\t',index_col=0)
met_forecast = pd.read_csv('/content/drive/MyDrive/FYP/met-forecast.tsv',sep='\t',index_col=0) 
rl_sites = pd.read_csv('/content/drive/MyDrive/FYP/rl-sites.tsv',sep='\t',index_col=0)
rl_kpis = pd.read_csv('/content/drive/MyDrive/FYP/rl-kpis.tsv',sep='\t',index_col=0)
distances = pd.read_csv('/content/drive/MyDrive/FYP/distances.tsv',sep='\t', index_col=0)
met_real = pd.read_csv('/content/drive/MyDrive/FYP/met-real.tsv',nrows=3000, sep='\t',index_col=0)

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
met_stations.head()

,station_no,height,clutter_class
0,WS_17047,381,DENSE TREE
1,WS_17062,5,LOW-MEDIUM URBAN
2,WS_17063,99,AIRPORT
3,WS_17064,18,OPEN IN URBAN
4,WS_17065,123,LOW-SPARSE URBAN


In [ ]:
rl_kpis.head()

,type,datetime,tip,mlid,mw_connection_no,site_id,card_type,adaptive_modulation,freq_band,severaly_error_second,error_second,unavail_second,avail_time,bbe,rxlevmax,capacity,modulation,rlf
0,ENK,2018-12-31,FAR,A0BE,"1,349,988",RL_;ABDV,cardtype1,Enable,f3,0.0,0.0,0.0,86.4,0.0,-31.0,456.0,1024QAM,False
1,ENK,2018-12-31,FAR,A0BI,"1,349,988",RL_;ABDV,cardtype1,Enable,f3,0.0,0.0,0.0,86.4,0.0,-30.7,456.0,1024QAM,False
2,ENK,2018-12-31,FAR,A5AB,"1,344,018",RL_;ABDV,cardtype4,Enable,f3,0.0,0.0,0.0,86.4,0.0,-34.4,406.0,512QAM,False
3,NEC,2018-12-31,FAR,A8CQ,"1,351,204",RL_;ABDV,cardtype5,Enable,f2,0.0,0.0,0.0,86.4,0.0,-35.3,247.0,2048QAM*,False
4,NEC,2018-12-31,FAR,A8DQ,"1,351,204",RL_;ABDV,cardtype5,Enable,f2,0.0,0.0,0.0,86.4,0.0,-35.3,247.0,2048QAM*,False


In [ ]:
rl_kpis['adaptive_modulation'].value_counts()

Enable     1798492
Disable     194494
Name: adaptive_modulation, dtype: int64

In [ ]:
met_forecast.head()

,station_no,datetime,report_time,weather_day1,temp_max_day1,temp_min_day1,humidity_max_day1,humidity_min_day1,wind_dir_day1,wind_speed_day1,weather_day2,temp_max_day2,temp_min_day2,humidity_max_day2,humidity_min_day2,wind_dir_day2,wind_speed_day2,weather_day3,temp_max_day3,temp_min_day3,humidity_max_day3,humidity_min_day3,wind_dir_day3,wind_speed_day3,weather_day4,temp_max_day4,temp_min_day4,humidity_max_day4,humidity_min_day4,wind_dir_day4,wind_speed_day4,weather_day5,temp_max_day5,temp_min_day5,humidity_max_day5,humidity_min_day5,wind_dir_day5,wind_speed_day5
0,WS_17047,2018-01-02,evening,heavy rain,10,6,96.0,82.0,6.0,19.0,rain,9,5,96.0,78.0,5.0,18.0,overcast clouds,10,2,93.0,69.0,213.0,14.0,few clouds,12,4,92.0,57.0,185.0,12.0,few clouds,14,8,90.0,70.0,210.0,16.0
1,WS_17047,2018-01-02,morning,heavy rain,10,6,96.0,82.0,6.0,19.0,rain,9,5,96.0,78.0,5.0,18.0,overcast clouds,10,2,93.0,69.0,213.0,14.0,few clouds,12,4,92.0,57.0,185.0,12.0,few clouds,14,8,90.0,70.0,210.0,16.0
2,WS_17062,2018-01-02,evening,heavy rain,12,9,96.0,80.0,84.0,10.0,rain,12,8,96.0,80.0,24.0,14.0,overcast clouds,12,6,89.0,75.0,228.0,16.0,few clouds,14,7,86.0,57.0,114.0,10.0,few clouds,17,10,78.0,67.0,107.0,14.0
3,WS_17062,2018-01-02,morning,heavy rain,12,9,96.0,80.0,84.0,10.0,rain,12,8,96.0,80.0,24.0,14.0,overcast clouds,12,6,89.0,75.0,228.0,16.0,few clouds,14,7,86.0,57.0,114.0,10.0,few clouds,17,10,78.0,67.0,107.0,14.0
4,WS_17063,2018-01-02,evening,heavy rain,13,9,95.0,78.0,87.0,10.0,rain,13,7,95.0,75.0,27.0,14.0,overcast clouds,11,7,92.0,77.0,14.0,14.0,few clouds,14,6,86.0,57.0,134.0,11.0,few clouds,17,10,78.0,63.0,131.0,14.0


In [ ]:
rl_sites.head()

,site_id,groundheight,clutter_class
0,RL_;ABDV,107.9658,OPEN LAND
1,RL_;ABL>,97.9733,OPEN LAND
2,RL_;ABTP,3.0446,OPEN LAND
3,RL_;AC?E,85.0143,OPEN LAND
4,RL_;ADKP,192.9800,DENSE TREE


In [ ]:
# Station names
stations = met_stations['station_no'].tolist()

In [ ]:
# Forecast dates
forecast_dates = sorted(list(set(met_forecast['datetime'])))

In [ ]:
met_forecast.sort_values(by=['station_no', 'datetime']).head()

,station_no,datetime,report_time,weather_day1,temp_max_day1,temp_min_day1,humidity_max_day1,humidity_min_day1,wind_dir_day1,wind_speed_day1,weather_day2,temp_max_day2,temp_min_day2,humidity_max_day2,humidity_min_day2,wind_dir_day2,wind_speed_day2,weather_day3,temp_max_day3,temp_min_day3,humidity_max_day3,humidity_min_day3,wind_dir_day3,wind_speed_day3,weather_day4,temp_max_day4,temp_min_day4,humidity_max_day4,humidity_min_day4,wind_dir_day4,wind_speed_day4,weather_day5,temp_max_day5,temp_min_day5,humidity_max_day5,humidity_min_day5,wind_dir_day5,wind_speed_day5
0,WS_17047,2018-01-02,evening,heavy rain,10,6,96.0,82.0,6.0,19.0,rain,9,5,96.0,78.0,5.0,18.0,overcast clouds,10,2,93.0,69.0,213.0,14.0,few clouds,12,4,92.0,57.0,185.0,12.0,few clouds,14,8,90.0,70.0,210.0,16.0
1,WS_17047,2018-01-02,morning,heavy rain,10,6,96.0,82.0,6.0,19.0,rain,9,5,96.0,78.0,5.0,18.0,overcast clouds,10,2,93.0,69.0,213.0,14.0,few clouds,12,4,92.0,57.0,185.0,12.0,few clouds,14,8,90.0,70.0,210.0,16.0
20,WS_17047,2018-01-03,evening,light rain,9,4,96.0,85.0,6.0,19.0,scattered clouds,9,3,92.0,59.0,5.0,19.0,few clouds,10,2,95.0,65.0,194.0,11.0,clear sky,13,5,91.0,67.0,210.0,16.0,clear sky,14,8,94.0,78.0,211.0,16.0
21,WS_17047,2018-01-03,morning,light rain,9,4,96.0,85.0,6.0,19.0,scattered clouds,9,3,92.0,59.0,5.0,19.0,few clouds,10,2,95.0,65.0,194.0,11.0,clear sky,13,5,91.0,67.0,210.0,16.0,clear sky,14,8,94.0,78.0,211.0,16.0
40,WS_17047,2018-01-04,evening,scattered clouds,9,4,91.0,60.0,0.0,16.0,few clouds,10,3,96.0,66.0,190.0,9.0,foggy,12,5,93.0,63.0,207.0,14.0,foggy,13,7,93.0,76.0,216.0,15.0,foggy,12,9,96.0,81.0,213.0,22.0


In [ ]:
to_drop = [c for c in met_forecast.columns 
           if ('day1' not in c) and (c not in ['station_no', 'datetime', 'report_time'])]
met_forecast_v2 = met_forecast.drop(columns=to_drop)
met_forecast_v2.head()

,station_no,datetime,report_time,weather_day1,temp_max_day1,temp_min_day1,humidity_max_day1,humidity_min_day1,wind_dir_day1,wind_speed_day1
0,WS_17047,2018-01-02,evening,heavy rain,10,6,96.0,82.0,6.0,19.0
1,WS_17047,2018-01-02,morning,heavy rain,10,6,96.0,82.0,6.0,19.0
2,WS_17062,2018-01-02,evening,heavy rain,12,9,96.0,80.0,84.0,10.0
3,WS_17062,2018-01-02,morning,heavy rain,12,9,96.0,80.0,84.0,10.0
4,WS_17063,2018-01-02,evening,heavy rain,13,9,95.0,78.0,87.0,10.0


In [ ]:
mean_values = met_forecast_v2.groupby(by=['station_no', 'datetime']).mean().reset_index()
mean_values.sort_values(by=['station_no', 'datetime']).head()

,station_no,datetime,temp_max_day1,temp_min_day1,humidity_max_day1,humidity_min_day1,wind_dir_day1,wind_speed_day1
0,WS_17047,2018-01-02,10.0,6.0,96.0,82.0,6.0,19.0
1,WS_17047,2018-01-03,9.0,4.0,96.0,85.0,6.0,19.0
2,WS_17047,2018-01-04,9.0,4.0,91.0,60.0,0.0,16.0
3,WS_17047,2018-01-05,10.0,3.0,96.0,70.0,194.0,9.0
4,WS_17047,2018-01-06,11.0,4.0,96.0,63.0,209.0,14.0


**Categorical data**

In [ ]:
numerical_values = met_forecast_v2.describe().columns
non_num_df = met_forecast_v2[[x for x in met_forecast_v2.columns 
                              if x not in numerical_values]]
non_num_df.head()

,station_no,datetime,report_time,weather_day1
0,WS_17047,2018-01-02,evening,heavy rain
1,WS_17047,2018-01-02,morning,heavy rain
2,WS_17062,2018-01-02,evening,heavy rain
3,WS_17062,2018-01-02,morning,heavy rain
4,WS_17063,2018-01-02,evening,heavy rain


In [ ]:
# 'Report_time' will be dropped
ohe_df = copy.deepcopy(non_num_df[['station_no', 'datetime']])
for x in range(1):
  temp = pd.get_dummies(non_num_df[f'weather_day{x+1}'], prefix=f'wd{x+1}')
  ohe_df = pd.concat([ohe_df, temp], axis=1)
ohe_df.head()

,station_no,datetime,wd1_clear sky,wd1_few clouds,wd1_foggy,wd1_heavy rain,wd1_heavy rain showers,wd1_heavy thunderstorm with rain showers,wd1_hot day,wd1_light intensity shower rain,wd1_light rain,wd1_light rain showers,wd1_light snow,wd1_misty,wd1_overcast clouds,wd1_rain,wd1_scattered clouds,wd1_sleet,wd1_snow,wd1_thunderstorm with heavy rain,wd1_windy
0,WS_17047,2018-01-02,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,WS_17047,2018-01-02,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,WS_17062,2018-01-02,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,WS_17062,2018-01-02,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,WS_17063,2018-01-02,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Dealing with two datetimes
ohe_df = ohe_df.groupby(by=['station_no', 'datetime'], as_index=False).agg(lambda x: 1 if sum(x) > 1 else sum(x))
ohe_df.head()

,station_no,datetime,wd1_clear sky,wd1_few clouds,wd1_foggy,wd1_heavy rain,wd1_heavy rain showers,wd1_heavy thunderstorm with rain showers,wd1_hot day,wd1_light intensity shower rain,wd1_light rain,wd1_light rain showers,wd1_light snow,wd1_misty,wd1_overcast clouds,wd1_rain,wd1_scattered clouds,wd1_sleet,wd1_snow,wd1_thunderstorm with heavy rain,wd1_windy
0,WS_17047,2018-01-02,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,WS_17047,2018-01-03,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,WS_17047,2018-01-04,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,WS_17047,2018-01-05,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,WS_17047,2018-01-06,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


**Combining the numerical data with the categorical**

In [ ]:
modified_forecast_df = pd.merge(left=mean_values, right=ohe_df, on=['station_no', 'datetime'])
modified_forecast_df.head()

,station_no,datetime,temp_max_day1,temp_min_day1,humidity_max_day1,humidity_min_day1,wind_dir_day1,wind_speed_day1,wd1_clear sky,wd1_few clouds,wd1_foggy,wd1_heavy rain,wd1_heavy rain showers,wd1_heavy thunderstorm with rain showers,wd1_hot day,wd1_light intensity shower rain,wd1_light rain,wd1_light rain showers,wd1_light snow,wd1_misty,wd1_overcast clouds,wd1_rain,wd1_scattered clouds,wd1_sleet,wd1_snow,wd1_thunderstorm with heavy rain,wd1_windy
0,WS_17047,2018-01-02,10.0,6.0,96.0,82.0,6.0,19.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,WS_17047,2018-01-03,9.0,4.0,96.0,85.0,6.0,19.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,WS_17047,2018-01-04,9.0,4.0,91.0,60.0,0.0,16.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,WS_17047,2018-01-05,10.0,3.0,96.0,70.0,194.0,9.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,WS_17047,2018-01-06,11.0,4.0,96.0,63.0,209.0,14.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


On RL KPIS
-> Using modified data  \
-> Adding the forecast information from the nearest weather stations (from the previous day)\
-> Adding the KPI information from the previous day. 

In [ ]:
distances

,WS_19111,WS_17047,WS_18397,WS_17062,WS_17813,WS_17064,WS_19112,WS_17063,WS_18736,WS_17065,WS_18399,WS_17610,WS_18792,WS_18735,WS_18100,WS_17437,WS_17448,WS_18403,WS_18404,WS_17389,RL_U7MPL,RL_X;ORF,RL_X;OE?,RL_\K[RM,RL_UBUGK,RL_X;OQA,RL_\K[LJ,RL_L=SKK,RL_\K[QH,RL_\K[EJ,RL_\K[EH,RL_KMLM0,RL_KMBTO,RL_KMPPE,RL_[EDCP,RL_K=LMV,RL_SEI?I,RL_IBLMM,RL_JETKO,RL_b;ZI>,...,RL_[KBQ>,RL_S7EPE,RL_P7ZEQ,RL_S7ECS,RL_JETRB,RL_S7EA>,RL_UBUQQ,RL_S7EWB,RL_S7EL>,RL_SKDRB,RL_I9UCH,RL_K7EKV,RL_\;EFV,RL_S?[@@,RL_POOSQ,RL_JETQ@,RL_I9JER,RL_UENSQ,RL_SE[Q@,RL_\;EDF,RL_SHUIL,RL_N8BJ>,RL_[7TIQ,RL_U?OCQ,RL_X;O@Q,RL_SHU@L,RL_S:ISI,RL_J7DG?,RL_JET@P,RL_N;OEL,RL_]=UC>,RL_]ITCK,RL_[KBDF,RL_X;O@O,RL_b?LMI,RL_]IC?O,RL_S:PQD,RL_JEZWL,RL_LKEII,RL_[EOMJ
WS_19111,0.00000,18.65248,60.37075,41.38519,44.21637,53.29280,57.28976,63.45174,62.78558,54.45858,58.62208,79.43958,78.35707,96.01031,65.61676,63.28520,66.61600,48.13006,41.71827,39.38671,50.96124,63.36847,72.61144,68.53121,51.29432,59.53805,70.86277,85.26848,65.94984,63.78482,64.86733,45.29888,40.55249,66.11638,42.13462,42.30116,47.21409,41.88481,47.79698,40.80230,...,46.29812,41.55173,40.96884,42.46770,47.88025,41.63500,51.29432,41.71827,41.80154,48.29660,42.63424,44.88253,40.80230,43.13386,40.88557,48.29660,42.55097,41.63500,46.96428,40.80230,56.29052,43.88329,45.88177,45.54869,59.45478,57.70611,41.96808,41.30192,47.63044,43.71675,56.29052,39.88633,46.79774,59.53805,42.80078,41.30192,41.46846,47.79698,52.29356,42.63424
WS_17047,18.65248,0.00000,73.44414,57.53957,60.28748,69.69699,73.02779,79.27304,76.94148,69.61372,73.02779,90.34795,91.26392,107.50157,82.27076,79.68939,83.35327,62.95212,57.03995,56.79014,67.19889,79.02323,87.01715,83.85289,67.36543,75.52589,86.35099,98.75822,82.10422,80.35555,81.35479,56.29052,56.54033,81.85441,58.20573,57.20649,63.11866,57.20649,64.03463,56.54033,...,62.61904,57.62284,56.95668,58.45554,63.86809,57.70611,67.69851,57.78938,57.70611,64.53425,58.37227,61.12018,56.79014,59.28824,56.87341,64.53425,58.20573,57.95592,62.86885,56.29052,72.77798,60.12094,62.20269,61.53653,75.94224,74.02703,58.20573,57.03995,63.86809,59.87113,72.02855,55.54109,63.03539,76.02551,58.87189,56.87341,57.53957,63.78482,67.44870,58.62208
WS_18397,60.37075,73.44414,0.00000,23.48214,21.31712,19.65172,15.23841,16.82054,6.57833,11.65780,7.16122,21.40039,17.98632,36.13918,23.31560,21.40039,24.64792,14.15590,20.48442,30.22701,18.90229,15.90457,14.98860,16.23765,18.23613,17.07035,18.73575,25.31408,20.31788,22.64944,21.98328,18.40267,23.23233,16.82054,22.48290,19.31864,18.56921,20.48442,20.15134,22.39963,...,21.06731,22.98252,22.89925,22.06655,18.98556,22.98252,20.23461,22.73271,22.31636,19.73499,21.23385,21.65020,23.14906,22.31636,22.81598,20.15134,20.81750,23.56541,18.81902,21.65020,20.15134,21.98328,21.23385,19.98480,20.15134,18.90229,23.23233,21.73347,20.31788,21.98328,15.07187,22.73271,20.48442,20.40115,21.98328,21.65020,22.81598,18.98556,12.82358,21.98328
WS_17062,41.38519,57.53957,23.48214,0.00000,2.83118,12.15742,15.90457,22.06655,23.23233,13.98936,18.98556,44.96580,39.96960,59.12170,24.64792,22.14982,25.81370,9.40951,4.57985,7.16122,9.65932,21.98328,32.30876,27.31256,9.90913,18.15286,29.56085,45.79850,24.64792,22.81598,23.81522,20.98404,1.58213,24.81446,0.99924,6.16198,6.07871,4.33004,6.49506,2.49810,...,4.99620,0.66616,1.41559,1.49886,6.57833,0.58289,10.15894,0.83270,1.41559,6.99468,2.41483,3.58061,1.33232,1.74867,1.66540,6.99468,2.91445,0.41635,5.74563,3.74715,15.23841,2.58137,4.57985,4.33004,18.40267,16.40419,0.66616,2.83118,6.24525,2.33156,14.98860,3.49734,5.49582,18.40267,1.66540,2.99772,0.91597,6.41179,11.90761,1.49886
WS_17813,44.21637,60.28748,21.31712,2.83118,0.00000,9.49278,13.07339,19.23537,20.65096,11.24145,16.40419,42.71751,37.38823,56.62360,21.98328,19.48518,23.14906,7.41103,4.99620,8.90989,6.91141,19.15210,29.47758,24.48138,7.07795,15.32168,26.72967,43.05059,21.81674,20.15134,21.06731,20.98404,3.83042,21.98328,2.08175,6.24525,3.24753,4.66312,3.74715,4.08023,...,2.49810,2.66464,3.33080,1.83194,3.66

In [ ]:
distances[['WS_19111']]

,WS_19111
WS_19111,0.00000
WS_17047,18.65248
WS_18397,60.37075
WS_17062,41.38519
WS_17813,44.21637
...,...
RL_]IC?O,41.30192
RL_S:PQD,41.46846
RL_JEZWL,47.79698
RL_LKEII,52.29356


# **RL_LPIS_MOD**

In [ ]:
rl_sites = pd.read_csv('/content/drive/MyDrive/FYP/rl-sites.tsv',sep='\t')
rl_kpis_mod = pd.read_csv('/content/drive/MyDrive/FYP/rl-kpis.tsv', sep='\t')

In [ ]:
rl_kpis_mod.head()

,Unnamed: 0,type,datetime,tip,mlid,mw_connection_no,site_id,card_type,adaptive_modulation,freq_band,severaly_error_second,error_second,unavail_second,avail_time,bbe,rxlevmax,capacity,modulation,rlf
0,0,ENK,2018-12-31,FAR,A0BE,"1,349,988",RL_;ABDV,cardtype1,Enable,f3,0.0,0.0,0.0,86.4,0.0,-31.0,456.0,1024QAM,False
1,1,ENK,2018-12-31,FAR,A0BI,"1,349,988",RL_;ABDV,cardtype1,Enable,f3,0.0,0.0,0.0,86.4,0.0,-30.7,456.0,1024QAM,False
2,2,ENK,2018-12-31,FAR,A5AB,"1,344,018",RL_;ABDV,cardtype4,Enable,f3,0.0,0.0,0.0,86.4,0.0,-34.4,406.0,512QAM,False
3,3,NEC,2018-12-31,FAR,A8CQ,"1,351,204",RL_;ABDV,cardtype5,Enable,f2,0.0,0.0,0.0,86.4,0.0,-35.3,247.0,2048QAM*,False
4,4,NEC,2018-12-31,FAR,A8DQ,"1,351,204",RL_;ABDV,cardtype5,Enable,f2,0.0,0.0,0.0,86.4,0.0,-35.3,247.0,2048QAM*,False


In [ ]:
from datetime import datetime

In [ ]:
# Forecast datetime should be - 1 day from the kpis datetime
rl_kpis_mod['forecast_datetime'] = [datetime.strptime(x.replace('-','/'),'%Y/%m/%d')  - pd.Timedelta(days=1) for x in rl_kpis_mod['datetime']]

CPU times: user 1min 19s, sys: 430 ms, total: 1min 20s
Wall time: 1min 21s


In [ ]:
# To find nearest station
def find_nearest_stations(site_id: str, distances: pd.DataFrame, 
                          stations: list, k: int = 1) -> str:
  temp = distances[[site_id]].sort_values(by=[site_id])
  temp = temp.loc[[x for x in temp.index if x in stations]].head(k)
  return list(temp.index)

In [ ]:
# Getting nearest station (just 1) based on the antennas - This might take a while
rl_kpis_mod['nearest_station'] = [find_nearest_stations(site_id, distances, stations)[0] for site_id in rl_kpis_mod['site_id']]

In [ ]:
rl_kpis_mod

,Unnamed: 0,type,datetime,tip,mlid,mw_connection_no,site_id,card_type,adaptive_modulation,freq_band,severaly_error_second,error_second,unavail_second,avail_time,bbe,rxlevmax,capacity,modulation,rlf,forecast_datetime,nearest_station
0,0,ENK,2018-12-31,FAR,A0BE,"1,349,988",RL_;ABDV,cardtype1,Enable,f3,0.0,0.0,0.0,86.4,0.0,-31.0,456.0,1024QAM,False,2018-12-30,WS_18403
1,1,ENK,2018-12-31,FAR,A0BI,"1,349,988",RL_;ABDV,cardtype1,Enable,f3,0.0,0.0,0.0,86.4,0.0,-30.7,456.0,1024QAM,False,2018-12-30,WS_18403
2,2,ENK,2018-12-31,FAR,A5AB,"1,344,018",RL_;ABDV,cardtype4,Enable,f3,0.0,0.0,0.0,86.4,0.0,-34.4,406.0,512QAM,False,2018-12-30,WS_18403
3,3,NEC,2018-12-31,FAR,A8CQ,"1,351,204",RL_;ABDV,cardtype5,Enable,f2,0.0,0.0,0.0,86.4,0.0,-35.3,247.0,2048QAM*,False,2018-12-30,WS_18403
4,4,NEC,2018-12-31,FAR,A8DQ,"1,351,204",RL_;ABDV,cardtype5,Enable,f2,0.0,0.0,0.0,86.4,0.0,-35.3,247.0,2048QAM*,False,2018-12-30,WS_18403
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1992981,1992981,NEC,2020-12-25,NEAR,A6FD,"1,371,370",RL_bKZCQ,cardtype5,Enable,f3,0.0,0.0,0.0,86.4,0.0,-36.8,495.0,2048QAM*,False,2020-12-24,WS_17064
1992982,1992982,NEC,2020-12-25,NEAR,A6NA,"1,371,370",RL_bKZCQ,cardtype5,Enable,f3,0.0,0.0,0.0,86.4,0.0,-36.8,495.0,2048QAM*,False,2020-12-24,WS_17064
1992983,1992983,NEC,2020-12-25,FAR,A8FJ,"1,371,440",RL_bKZCQ,cardtype5,Enable,f3,0.0,0.0,0.0,86.4,0.0,-28.9,495.0,2048QAM*,False,2020-12-24,WS_17064
1992984,1992984,NEC,2020-12-25,FAR,A8HV,"1,371,440",RL_bKZCQ,cardtype5,Enable,f3,0.0,0.0,0.0,86.4,0.0,-28.9,495.0,2048QAM*,False,2020-12-24,WS_17064


In [ ]:
rl_kpis_mod.to_csv('rl_kpis_mod.csv')

In [ ]:
rl_kpis_mod= pd.read_csv('/content/drive/MyDrive/FYP/rl_kpis_mod.csv', index_col=0)

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
